<a href="https://colab.research.google.com/github/amitkag85/AILearning/blob/main/populate_vector_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai ipywidgets pandas SQLAlchemy nltk

Imports

In [ ]:
import json
import ipywidgets as widgets
from IPython.display import display
from openai import AzureOpenAI
import psycopg2
import nltk
from tqdm import tqdm

Inputs for all secrets.

In [ ]:
openai_api_url = widgets.Text(
    value='',
    placeholder='Enter OpenAI Embedding API URL',
    description='Emb URL:',
    disabled=False
)

display(openai_api_url)

openai_api_key = widgets.Password(
    value='',
    placeholder='Enter OpenAI Embedding API Key',
    description='Emb API Key:',
    disabled=False
)

display(openai_api_key)

db_host = widgets.Text(
    value='',
    placeholder='Enter Host Name',
    description='Host:',
    disabled=False
)

display(db_host)

db_name = widgets.Text(
    value='',
    placeholder='Enter Name',
    description='Database:',
    disabled=False
)

display(db_name)

db_admin = widgets.Text(
    value='',
    placeholder='Enter Admin',
    description='Admin:',
    disabled=False
)

display(db_admin)

db_password = widgets.Password(
    value='',
    placeholder='Enter Database Password',
    description='Password:',
    disabled=False
)

display(db_password)

Wrapper to make database connections.

In [ ]:
def connection():
    return psycopg2.connect(
        host=db_host.value,
        database=db_name.value,
        user=db_admin.value,
        password=db_password.value
    )

Wrapper for calling the OpenAI embedding model.

In [ ]:
# create the client object with the required values
client = AzureOpenAI(
    api_key=openai_api_key.value,
    api_version="2024-02-01",
    azure_endpoint=openai_api_url.value
)

# model name from deployment
model_name = "text-embedding-3-small"

def embed(text, dimensions=None):
    response = client.embeddings.create(
        input=text,
        model=model_name,
        dimensions=dimensions
    )

    embedding = response.data[0].embedding
    return embedding

For sentence splitting.

In [ ]:
nltk.download('punkt')
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

Mount Google Drive to read file.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Path to data on Google Drive. Download file from [here](https://drive.google.com/file/d/1m8Z9lPGYmca61Z8m7q0Dp3o8lVuTUscr/view?usp=sharing), save it to your Google Drive, and copy the path here.

In [ ]:
TRANSCRIPT_PATH = '/content/drive/MyDrive/Pluralsight/Using ChatGPT in Python for NLP Applications in Azure/KPMG 10 1-10 24/SSTK_transcripts.json'  #@param {type:"string"}

Load data file.

In [ ]:
with open(TRANSCRIPT_PATH, 'r') as f:
  data = json.load(f)

print(data)

Print a list of all quarters in the data.

In [ ]:
quarters = sorted(list(data.keys()))
print(quarters)

Helper function for adding records for each sentence in a text block.

In [ ]:
def make_record(ticker, quarter_id, quarter, block_id, speaker, text):
  out = []
  for sentence_id, sentence in enumerate(sent_detector.tokenize(text)):
    out.append({'ticker': ticker,
                'quarter_id': quarter_id,
                'quarter': quarter,
                'block_id': block_id,
                'speaker': speaker,
                'sentence_id': sentence_id,
                'sentence': sentence})
  return out

Process all records.

In [ ]:
records = []
for quarter_id, quarter in enumerate(quarters):
  for block_id, block in enumerate(data[quarter]['transcript']):
    records.extend(make_record('SSTK',
                               quarter_id,
                               quarter,
                               block_id,
                               block['speaker'],
                               block['text']))

len(records)

Define the dimension of the embeddings (we will use less than the default).

In [ ]:
EMBEDDING_DIM = 768

Process all sentences with OpenAI (this takes a while).

In [ ]:
for record in tqdm(records):
  record['embedding'] = embed(record['sentence'], EMBEDDING_DIM)

If you have not yet created the `vector` extension, uncomment this block and run it.

In [ ]:
'''
with connection() as conn:
    cur = conn.cursor()
    cur.execute("SELECT CREATE_EXTENSION('vector')")
    conn.commit()
'''

Create the database table.

In [ ]:
with connection() as conn:
    cur = conn.cursor()
    cur.execute("""CREATE TABLE IF NOT EXISTS transcripts (
        id SERIAL PRIMARY KEY,
        ticker TEXT,
        quarter_id INTEGER,
        quarter TEXT,
        block_id INTEGER,
        speaker TEXT,
        sentence_id INTEGER,
        sentence TEXT,
        embedding vector({dim})
    )""".format(dim=EMBEDDING_DIM))
    conn.commit()

Insert all records into the database.

In [ ]:
with connection() as conn:
    cur = conn.cursor()
    for data in tqdm(records):
        cur.execute("""INSERT INTO transcripts
(
    ticker,
    quarter_id,
    quarter,
    block_id,
    speaker,
    sentence_id,
    sentence,
    embedding
) VALUES (
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s
)""", (data['ticker'],
       data['quarter_id'],
       data['quarter'],
       data['block_id'],
       data['speaker'],
       data['sentence_id'],
       data['sentence'],
       data['embedding'])
    )
    conn.commit()

Get a count of all inserted records as a sanity check.

In [ ]:
with connection() as conn:
    cur = conn.cursor()
    cur.execute("SELECT COUNT(*) FROM transcripts")
    count = cur.fetchone()[0]
    print(count)


Print some values from the first 10 records.

In [ ]:
with connection() as conn:
    cur = conn.cursor()
    cur.execute("""SELECT
    id, ticker, quarter, speaker, sentence
FROM transcripts
LIMIT 10""")
    rows = cur.fetchall()

for row in rows:
    print(row)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

Get some values for the first 10 records as a pandas dataframe.

In [ ]:
query = 'AI'
query_vector = embed(query, EMBEDDING_DIM)

engine = create_engine(f"postgresql://{db_admin.value}:{db_password.value}@{db_host.value}:5432/{db_name.value}")

df = pd.read_sql("""SELECT
    id,
    ticker,
    quarter,
    speaker,
    sentence,
    (1 - (embedding <=> '{emb}')) AS similarity
FROM transcripts
ORDER BY similarity DESC
LIMIT 10
""".format(emb=json.dumps(query_vector)), engine)

df
